In [ ]:
import numpy as np
import scipy.io
from scipy.signal import butter, filtfilt, hilbert
from sklearn.decomposition import FastICA, PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
import os
import glob

In [ ]:
# 滤波函数
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data, axis=-1)
    return y

In [ ]:
# ICA去噪函数
def apply_ica(data, n_components=3):
    ica = FastICA(n_components=n_components)
    cleaned_data = ica.fit_transform(data)
    return cleaned_data

In [ ]:
# 计算PLV特征函数
def compute_plv(data):
    n_samples, n_channels, n_times = data.shape
    plv = np.zeros((n_channels, n_channels))

    for i in range(n_channels):
        for j in range(i + 1, n_channels):
            phase_diff = np.angle(hilbert(data[:, i, :])) - np.angle(hilbert(data[:, j, :]))
            plv[i, j] = np.abs(np.sum(np.exp(1j * phase_diff), axis=-1)) / n_times

    return plv

In [ ]:
# PCA降维函数
def apply_pca(data, n_components=10):
    pca = PCA(n_components=n_components)
    reduced_data = pca.fit_transform(data.reshape(data.shape[0], -1))
    return reduced_data


In [ ]:
# 训练LDA模型并进行交叉验证
def train_lda(X, y):
    lda = LDA()
    skf = StratifiedKFold(n_splits=10)
    scores = cross_val_score(lda, X, y, cv=skf)
    print(f'Cross-validation accuracy: {np.mean(scores):.2f}')
    return lda.fit(X, y)

In [ ]:
# 加载.mat文件数据
def load_mat_files(directory):
    files = glob.glob(os.path.join(directory, '*.mat'))
    all_data = []

    for file in files:
        mat_data = scipy.io.loadmat(file)
        for key in mat_data:
            if isinstance(mat_data[key], np.ndarray) and mat_data[key].dtype == 'float64':
                all_data.append(mat_data[key])

    return np.array(all_data)


In [ ]:
# 划分数据集
def split_dataset(data, labels, train_size=0.6, test_size=0.2, val_size=0.2):
    X_train, X_temp, y_train, y_temp = train_test_split(data, labels, train_size=train_size, random_state=42)
    test_ratio = test_size / (test_size + val_size)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=test_ratio, random_state=42)
    return X_train, X_test, X_val, y_train, y_test, y_val


In [ ]:
directory = 'path/to/mat/files'  # 替换为实际.mat文件目录
raw_data = load_mat_files(directory)

# 创建标签（假设每个文件有一个对应的标签）
labels = np.array([i // 15 for i in range(len(raw_data))])

# 划分数据集
X_train, X_test, X_val, y_train, y_test, y_val = split_dataset(raw_data, labels)

# 预处理
fs = 250  # 采样频率
lowcut = 8.0
highcut = 30.0

# 对训练数据进行滤波、去噪、特征提取和降维
filtered_data_train = bandpass_filter(X_train, lowcut, highcut, fs)
cleaned_data_train = apply_ica(filtered_data_train)
plv_features_train = compute_plv(cleaned_data_train)
reduced_data_train = apply_pca(plv_features_train, n_components=10)

# 对测试数据进行相同的处理
filtered_data_test = bandpass_filter(X_test, lowcut, highcut, fs)
cleaned_data_test = apply_ica(filtered_data_test)
plv_features_test = compute_plv(cleaned_data_test)
reduced_data_test = apply_pca(plv_features_test, n_components=10)

# 对验证数据进行相同的处理
filtered_data_val = bandpass_filter(X_val, lowcut, highcut, fs)
cleaned_data_val = apply_ica(filtered_data_val)
plv_features_val = compute_plv(cleaned_data_val)
reduced_data_val = apply_pca(plv_features_val, n_components=10)

# 训练LDA模型并进行交叉验证
lda_model = train_lda(reduced_data_train, y_train)

# 测试和验证
y_test_pred = lda_model.predict(reduced_data_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Test accuracy: {test_accuracy:.2f}')

y_val_pred = lda_model.predict(reduced_data_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation accuracy: {val_accuracy:.2f}')